# FOMO-AD Model For Anomaly Detection

## Introduction

## Before running the notebook

Let's prepare the environement before running the notebook.  
Run the following command in the terminal from the **`final`** folder.

```sh
conda create -p envs/3_fomoad python=3.11.7 -y
conda activate envs/3_fomoad

pip install -r requirements/3_fomoad.txt
```

## Login

In [64]:
import edgeimpulse_api as ei_api
import os
from dotenv import load_dotenv
import random  
import requests  
import json  

load_dotenv()

cfg = {
    "project_name": "FOMO-AD-cookies-1",
    "dataset_path": "datasets/cookies_1",
    "project_api_key": os.getenv("EDGE_IMPULSE_API_KEY_FOMO_AD_COOKIES_1"),
    "project_api_key_dev": os.getenv("EDGE_IMPULSE_API_KEY_FOMO_AD_COOKIES_1_DEV"),
    
    "output_path": "../output/fomoad_1.eim",
    
    # Only if you want to find the best FOMO config for the training
    # If you don't want, set the value to 0, it will train the model with default parameters
    # organization_api_key and pipeline_id don't need to be set if the value is 0 for organization_id
    "organization_id": 105154,
    "organization_api_key": os.getenv("EDGE_IMPULSE_API_KEY_ORGANISATION"),
    "pipeline_id": 3923 
}

In [ ]:
def login():
    url = "https://studio.edgeimpulse.com/v1/api-login"  
    payload = json.dumps({  
        "username": os.getenv("EDGE_IMPULSE_USERNAME"),
        "password": os.getenv("EDGE_IMPULSE_PASSWORD") 
    })  
    headers = {  
        'content-type': "application/json"
    }  
    
    response = requests.post(url, data=payload, headers=headers)
    res = response.json()

    print("Res", res)
    return res["token"]

token = login()

## Get active projects

In [3]:
def get_active_projects():
    response = requests.get("https://studio.edgeimpulse.com/v1/api/projects", headers={"x-jwt-token": token})
    res = response.json()
    print(res)
    return res["projects"]
    
projects = get_active_projects()

{'success': True, 'projects': [{'id': 368890, 'name': 'callmemath-pro-project-1', 'description': 'This is your Edge Impulse project. From here you acquire new training data, design impulses and train models.', 'created': '2024-03-22T10:12:59.638Z', 'lastAccessed': '2024-04-16T09:24:46.807Z', 'owner': 'math-pro', 'collaborators': [{'id': 175414, 'created': '2024-03-22T10:12:59.638Z', 'staffInfo': {'isStaff': False, 'hasSudoRights': False, 'isTestuser': False}, 'name': 'math-pro', 'username': 'callmemath-pro', 'email': 'math@lescaudron.com', 'photo': '', 'pending': False, 'isOwner': True, 'activated': True, 'mfaConfigured': False}], 'labelingMethod': 'single_label', 'metadata': {}, 'isEnterpriseProject': False, 'whitelabelId': None, 'tier': 'free', 'hasPublicVersion': False, 'isPublic': False, 'allowsLivePublicAccess': False, 'ownerIsDeveloperProfile': True, 'developerProfileUserId': 175414, 'ownerOrganizationId': 101592, 'indPauseProcessingSamples': False, 'publicProjectListed': True}, 

In [4]:
project = next((project for project in projects if project['name'] == cfg["project_name"]), None)

print("ID:", project["id"])
print("Name:", project["name"])
print("Description:", project["description"])

ID: 371766
Name: FOMO-AD-cookies-2
Description: Anomaly Detection model using FOMO AD on cookies dataset type 2


## Data upload

In [5]:
def upload_data(category, files, label):
    res = requests.post(
        f"https://ingestion.edgeimpulse.com/api/{category}/files", 
        headers={"x-api-key": cfg["project_api_key"], "x-label": label, "x-disallow-duplicates": "1"},
        # Creating the data payload for the request.
        files=(('data', (os.path.basename(i), open(
            i, 'rb'), 'image/jpg')) for i in files)
        )
    
    if (res.status_code == 200):
        print('Uploaded file(s) to Edge Impulse\n', res.status_code, res.content)
    else:
        print('Failed to upload file(s) to Edge Impulse\n',
          res.status_code, res.content)


dataset_path_good=f"{cfg['dataset_path']}/no_anomaly"
dataset_path_defect_1=f"{cfg['dataset_path']}/anomaly_lvl_1"
dataset_path_defect_2=f"{cfg['dataset_path']}/anomaly_lvl_2"
dataset_path_defect_3=f"{cfg['dataset_path']}/anomaly_lvl_3"

# Good cookies
all_good_cookies=[f"{dataset_path_good}/{f}" for f in os.listdir(dataset_path_good) if f.endswith('.jpg')]
random.seed(42)
random.shuffle(all_good_cookies)

train_good_cookies = all_good_cookies[:-20]
test_good_cookies = all_good_cookies[-20:]

print("train_good_cookies len", len(train_good_cookies))
print("test_good_cookies len", len(test_good_cookies))

print("Uploading good cookies training")
upload_data("training", train_good_cookies, "no anomaly") # "no anomaly" and "anomaly" is important!
print("Uploading good cookies testing")
upload_data("testing", test_good_cookies, "no anomaly")

all_defect_cookies_lvl_1=[f"{dataset_path_defect_1}/{f}" for f in os.listdir(dataset_path_defect_1) if f.endswith('.jpg')]
all_defect_cookies_lvl_2=[f"{dataset_path_defect_2}/{f}" for f in os.listdir(dataset_path_defect_2) if f.endswith('.jpg')]
all_defect_cookies_lvl_3=[f"{dataset_path_defect_3}/{f}" for f in os.listdir(dataset_path_defect_3) if f.endswith('.jpg')]
all_defect_cookies = all_defect_cookies_lvl_1 + all_defect_cookies_lvl_2 + all_defect_cookies_lvl_3  

print("all_defect_cookies", len(all_defect_cookies))

print("Uploading defect cookies testing")
upload_data("testing", all_defect_cookies, "anomaly")
print("Done")

train_good_cookies len 80
test_good_cookies len 20
Uploading good cookies training
Uploaded file(s) to Edge Impulse
 200 b'{"success":true,"files":[{"success":true,"projectId":371766,"sampleId":933288167,"fileName":"20240417_133252.jpg.4rivljks.json"},{"success":true,"projectId":371766,"sampleId":933288169,"fileName":"20240417_133534.jpg.4rivljq2.json"},{"success":true,"projectId":371766,"sampleId":933288171,"fileName":"20240417_133619.jpg.4rivlk06.json"},{"success":true,"projectId":371766,"sampleId":933288173,"fileName":"20240417_133159.jpg.4rivlk43.json"},{"success":true,"projectId":371766,"sampleId":933288175,"fileName":"20240417_133134.jpg.4rivlk7j.json"},{"success":true,"projectId":371766,"sampleId":933288177,"fileName":"20240417_133536.jpg.4rivlkco.json"},{"success":true,"projectId":371766,"sampleId":933288179,"fileName":"20240417_133403.jpg.4rivlkhd.json"},{"success":true,"projectId":371766,"sampleId":933288181,"fileName":"20240417_133647.jpg.4rivlkoa.json"},{"success":true,"pro

## Impulse

### Get impulse

In [6]:
def get_impulse(projectId):
    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/impulse", headers={"x-jwt-token": token})
    res = response.json()
    return res

# get_impulse(project_fomoad["id"])
get_impulse(project["id"])

{'success': True,
 'impulse': {'inputBlocks': [{'db': False,
    'id': 11,
    'name': 'Image data',
    'type': 'image',
    'title': 'Image data',
    'enabled': True,
    'createdBy': 'clone',
    'cropAnchor': 'middle-center',
    'imageWidth': 160,
    'resizeMode': 'squash',
    'description': 'EON Tuner Primary',
    'imageHeight': 160,
    'resizeMethod': 'lanczos3',
    'labelingMethod': 'single_label',
    'tunerTemplateId': -1,
    'createdAt': '2024-04-16T08:16:51.849Z',
    'mutated': False,
    'clonedFromBlockId': 4516308,
    'primaryVersion': True}],
  'dspBlocks': [{'db': False,
    'id': 12,
    'axes': ['image'],
    'name': 'Image',
    'type': 'image',
    'input': 11,
    'title': 'Image',
    'enabled': True,
    'autotune': False,
    'createdBy': 'clone',
    'description': 'EON Tuner Primary',
    'tunerTemplateId': -1,
    'implementationVersion': 1,
    'createdAt': '2024-04-16T08:16:51.849Z',
    'mutated': False,
    'clonedFromBlockId': 4516309,
    'pri

### Set Impulse

In [7]:
impulse_payload = {
    "inputBlocks": [
      {
        "db": False,
        "id": 11,
        "name": "Image data",
        "type": "image",
        "title": "Image data",
        "enabled": True,
        "createdBy": "clone",
        "cropAnchor": "middle-center",
        "imageWidth": 160,
        "resizeMode": "squash",
        "description": "EON Tuner Primary",
        "imageHeight": 160,
        "resizeMethod": "lanczos3",
        "labelingMethod": "single_label",
        # "primaryVersion": True,
        "tunerTemplateId": -1,
        "createdAt": "2024-04-16T08:16:51.849Z",
        "mutated": False,
        "clonedFromBlockId": 4516308,
        "tunerPrimary": True
      }
    ],
    "dspBlocks": [
      {
        "db": False,
        "id": 12,
        "axes": [
          "image"
        ],
        "name": "Image",
        "type": "image",
        "input": 11,
        "title": "Image",
        "enabled": True,
        "autotune": False,
        "createdBy": "clone",
        "description": "EON Tuner Primary",
        # "primaryVersion": True,
        "tunerTemplateId": -1,
        "implementationVersion": 1,
        "createdAt": "2024-04-16T08:16:51.849Z",
        "mutated": False,
        "clonedFromBlockId": 4516309,
        "tunerPrimary": True
      }
    ],
    "learnBlocks": [
      {
        "db": False,
        "id": 13,
        "dsp": [
          12
        ],
        "name": "FOMO-AD",
        "type": "keras-visual-anomaly",
        "title": "FOMO-AD (Images)",
        "enabled": True,
        "createdBy": "clone",
        "description": "EON Tuner Primary",
        "primaryVersion": True,
        "tunerTemplateId": -1,
        "createdAt": "2024-04-16T08:16:55.287Z",
        "mutated": True,
        "clonedFromBlockId": 4516328,
        "tunerPrimary": True
      }
    ]
  }


In [8]:

# DEFAULT LEARNING BLOCK
default_impulse_payload = {
    "inputBlocks": [
      {
        "db": False,
        "id": 11,
        "name": "Image data",
        "type": "image",
        "title": "Image data",
        "enabled": True,
        "createdBy": "clone",
        "cropAnchor": "middle-center",
        "imageWidth": 160,
        "resizeMode": "squash",
        "description": "EON Tuner Primary",
        "imageHeight": 160,
        "resizeMethod": "lanczos3",
        "labelingMethod": "single_label",
        # "primaryVersion": True,
        "tunerTemplateId": -1,
        "createdAt": "2024-04-16T08:16:51.849Z",
        "mutated": False,
        "clonedFromBlockId": 4516308,
        # "tunerPrimary": True
      }
    ],
    "dspBlocks": [
      {
        "db": False,
        "id": 12,
        "axes": [
          "image"
        ],
        "name": "Image",
        "type": "image",
        "input": 11,
        "title": "Image",
        "enabled": True,
        "autotune": False,
        "createdBy": "clone",
        "description": "EON Tuner Primary",
        # "primaryVersion": True,
        "tunerTemplateId": -1,
        "implementationVersion": 1,
        "createdAt": "2024-04-16T08:16:51.849Z",
        "mutated": False,
        "clonedFromBlockId": 4516309,
        # "tunerPrimary": True
      }
    ],
    "learnBlocks": [
      {
        "id": 15,
        "type": "keras-visual-anomaly",
        "name": "FOMO-AD",
        "dsp": [
          12
        ],
        "title": "FOMO-AD (Images)",
        "primaryVersion": True,
        "createdBy": "createImpulse",
        "createdAt": "2024-04-16T09:04:22.040Z"
      }
    ]
}

In [9]:
def delete_impulse(projectId):
    response = requests.delete(f"https://studio.edgeimpulse.com/v1/api/{projectId}/impulse", headers={"x-jwt-token": token})
    res = response.json()
    return res

def create_impulse(projectId, payload):
    delete_impulse(projectId)
    response = requests.post(f"https://studio.edgeimpulse.com/v1/api/{projectId}/impulse", json=payload, headers={"x-jwt-token": token})
    res = response.json()
    return res

create_impulse(project["id"], default_impulse_payload)
# create_impulse(project_fomoad["id"], impulse_payload)

{'success': True}

### Generate features

In [10]:
def get_jobs(projectId):
    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/all", headers={"x-jwt-token": token})
    res = response.json()
    return res

def get_job_status(projectId, jobId):
    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/{jobId}/status", headers={"x-jwt-token": token})
    res = response.json()
    return res["job"]

def get_job_output(projectId, jobId):
    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/{jobId}/stdout", headers={"x-jwt-token": token})
    res = response.json()
    return res

def generate_feature(projectId):
    
    payload = {
        "dspId": 12,
        "calculateFeatureImportance": False
    }
    response = requests.post(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/generate-features", json=payload, headers={"x-jwt-token": token})
    res = response.json()
    return res["id"]

In [11]:
job_id = generate_feature(project["id"])
print(job_id)

18483524


In [12]:
get_job_status(project["id"], job_id)

{'id': 18483524,
 'category': 'Generating features (Image)',
 'key': 'dsp-studio-wrapper-12',
 'created': '2024-04-17T15:33:30.865Z',
 'jobNotificationUids': [],
 'additionalInfo': 'Image',
 'computeTime': 0,
 'createdByUser': {'id': 175414,
  'name': 'math-pro',
  'username': 'callmemath-pro'}}

In [13]:
import time  

while True:  
    status = get_job_status(project["id"], job_id)  
    if "finishedSuccessful" in status and status["finishedSuccessful"]:
        print("Feature generated!") 
        break  
    if "finished" in status and status["finished"]:
        print("Generating done not successful", status) 
        break
    
    print("- Generating feature still in progress, waiting 10sec")
    time.sleep(10)

- Generating feature still in progress, waiting 10sec
- Generating feature still in progress, waiting 10sec
- Generating feature still in progress, waiting 10sec
- Generating feature still in progress, waiting 10sec
- Generating feature still in progress, waiting 10sec
Feature generated!


### Train Model

In [35]:
def train_model_keras(projectId, learnId):
    payload = {
        "trainTestSplit": 0.2,
        "customValidationMetadataKey": "",
        "autoClassWeights": False,
        "profileInt8": True,
        "learningRate": 0.01,
        "trainingCycles": 1,
        "visualLayers": [
            {
                "type": "transfer_mobilenetv2_a35"
            }
        ],
        "augmentationPolicyImage": "none",
        "anomalyCapacity": "low",
        "customParameters": {}
    }
    
    response = requests.post(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/train/keras/{learnId}", json=payload, headers={"x-jwt-token": token})
    res = response.json()
    return res["id"]

if cfg["organization_id"] == 0:
    job_id_train = train_model_keras(project["id"], default_impulse_payload["learnBlocks"][0]["id"])
    while True:
        status = get_job_status(project["id"], job_id_train)  
        if "finishedSuccessful" in status and status["finishedSuccessful"]:
            print("Training done") 
            break  
        if "finished" in status and status["finished"]:
            print("Training done not successful", status) 
            break  
        print("- Training is still in progress, waiting 10sec...")
        time.sleep(10)
else:
    print("Skipping training, will use transformation blocks instead")
    
# job_id_train


Skipping training, will use transformation blocks instead


In [65]:
# payload = {
#     "name": "Import data using \"Find best Visual AD model\"",
#     "description": "Runs transformation block 'Find best Visual AD model' to import data into this project",
#     "emailRecipientUids": [
#         175414
#     ],
#     "projectId": 371766,
#     "steps": [
#         {
#             "name": "Import data using Find best Visual AD model",
#             "transformationBlockId": 5671,
#             "extraCliArguments": "",
#             "parameters": {
#                 "api-key": "using anything else",
#                 "image-sizes": "96, 160, 224, 320",
#                 "model-types": "transfer_mobilenetv2_a35",
#                 "capacities": "low, medium, high",
#                 "image-resize-mode": "squash",
#                 "image-channels": "RGB"
#             }
#         },
#         {
#             "name": "Retrain model",
#             "builtinTransformationBlock": {
#                 "type": "project-action",
#                 "retrainModel": True
#             }
#         }
#     ],
#     "whenToEmail": "always"
# }

# def create_pipeline(payload):
#     response = requests.get(f"https://studio.edgeimpulse.com/v1/api/organizations/{cfg['organization_id']}/pipelines", headers={"x-jwt-token": token})
#     res = response.json()
#     return res

# if cfg["organization_id"] == 0:
#     pass
# else:
#     pipeline = create_pipeline(payload)
# pipeline

def run_pipeline():
    print(f"https://studio.edgeimpulse.com/v1/api/organizations/{cfg['organization_id']}/pipelines/{cfg['pipeline_id']}/run")
    response = requests.post(f"https://studio.edgeimpulse.com/v1/api/organizations/{cfg['organization_id']}/pipelines/{cfg['pipeline_id']}/run", headers={"x-api-key": cfg["organization_api_key"]})
    return response.json()



if cfg["organization_id"] == 0:
    pass
else:
    pipeline = run_pipeline()
pipeline

https://studio.edgeimpulse.com/v1/api/organizations/105154/pipelines/3928/run


{'success': True,
 'pipelineRun': {'id': 429023,
  'created': '2024-04-17T17:09:02.519Z',
  'steps': [{'name': 'Import data using "Find best Visual AD model" - Import data using Find best Visual AD model - 2024-04-17T17:09:02.448Z',
    'transformationJob': {'id': 576931,
     'organizationId': 105154,
     'name': 'Import data using "Find best Visual AD model" - Import data using Find best Visual AD model - 2024-04-17T17:09:02.448Z',
     'uploadJobStatus': 'waiting',
     'uploadType': 'project',
     'projectId': 0,
     'projectName': '',
     'transformationBlockId': 5671,
     'transformationBlockName': 'Find best Visual AD model',
     'category': 'training',
     'created': '2024-04-17T17:09:02.459Z',
     'totalDownloadFileCount': 0,
     'totalDownloadFileSize': 0,
     'totalDownloadFileSizeString': '0 Bytes',
     'totalUploadFileCount': 0,
     'transformationSummary': {'startedCount': 0,
      'finishedCount': 0,
      'succeededCount': 0,
      'totalFileCount': 0,
     

In [42]:
pipeline

{'success': True,
 'pipelines': [{'id': 3923,
   'description': "Runs transformation block 'Find best Visual AD model' to import data into this project",
   'name': 'Import data using "Find best Visual AD model"',
   'created': '2024-04-17T15:22:22.361Z',
   'steps': [{'name': 'Import data using Find best Visual AD model',
     'transformationBlockId': 5671,
     'extraCliArguments': '',
     'parameters': {'api-key': 'ei_ab9a6acba618869c64c553841bc9c9eb876a80f35cbfb4fb',
      'image-sizes': '96, 160, 224, 320',
      'model-types': 'transfer_mobilenetv2_a35',
      'capacities': 'low, medium, high',
      'image-resize-mode': 'squash',
      'image-channels': 'RGB'}},
    {'name': 'Retrain model',
     'builtinTransformationBlock': {'type': 'project-action',
      'retrainModel': True}},
    {'name': 'Create new version',
     'builtinTransformationBlock': {'type': 'project-action',
      'createVersion': True}}],
   'lastRun': {'id': 429002,
    'created': '2024-04-17T15:33:57.12747

## Export model

In [18]:
def get_deployment_targets(projectId):
    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/deployment/targets", headers={"x-jwt-token": token})
    res = response.json()
    return res["targets"]

def build_model(projectId, eim_type="runner-mac-x86_64"):
    payload = {
       "engine": "tflite"
    }
    response = requests.post(f"https://studio.edgeimpulse.com/v1/api/{projectId}/jobs/build-ondevice-model?type={eim_type}", json=payload, headers={"x-jwt-token": token})
    res = response.json()
    return res["id"]

def download_model_built(projectId, eim_type="runner-mac-x86_64"):
    payload = {
       "engine": "tflite"
    }
    response = requests.get(f"https://studio.edgeimpulse.com/v1/api/{projectId}/deployment/download?type={eim_type}", headers={"x-jwt-token": token})
    return response


In [19]:
# targets = get_deployment_targets(project_to_use["id"])
# for item in targets:  
#     print(item['name'],"=",item["format"])

job_id_build = build_model(project["id"])

In [20]:
get_job_status(project["id"], job_id_build)  

{'id': 18483630,
 'category': 'Building deployment (runner-mac-x86_64)',
 'key': 'deployment-studio-wrapper',
 'created': '2024-04-17T15:35:34.389Z',
 'jobNotificationUids': [],
 'additionalInfo': 'runner-mac-x86_64',
 'computeTime': 0,
 'createdByUser': {'id': 175414,
  'name': 'math-pro',
  'username': 'callmemath-pro'}}

In [21]:
while True:
    status = get_job_status(project["id"], job_id_build)  
    if "finishedSuccessful" in status and status["finishedSuccessful"]:
        print("Building done") 
        break  
    if "finished" in status and status["finished"]:
        print("Building done not successful", status) 
        break  
    
    print("- Building is still in progress, waiting 10sec...")
    time.sleep(10)

- Building is still in progress, waiting 10sec...
- Building is still in progress, waiting 10sec...
- Building is still in progress, waiting 10sec...
- Building is still in progress, waiting 10sec...
Building done


In [25]:
data = download_model_built(project["id"])
with open(cfg["output_path"], 'wb') as f:
    f.write(data.content)

In [26]:
# For the macos model only
!chmod +x "../output/fomoad_1.eim"

In [29]:
from edge_impulse_linux.image import ImageImpulseRunner
import cv2  

model = cfg["output_path"]
modelfile = os.path.join(model)
print("modelfile", modelfile)

def ei_inference(img_path):
    with ImageImpulseRunner(modelfile) as runner:
        model_info = runner.init()
        # print("model_info", model_info)
        
        # Load the image directly from the disk  
        original_image = cv2.imread(img_path, cv2.IMREAD_COLOR)  
        # Convert the image from BGR to RGB (since OpenCV loads images in BGR format)  
        img = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)  
        
        features, cropped = runner.get_features_from_image(img)
        # print("FEATURES", len(features), len(cropped))
        
        # print("GO")
        res = runner.classify(features)
        print(res["result"])



all_defect_cookies_lvl_1=[f"{dataset_path_defect_1}/{f}" for f in os.listdir(dataset_path_defect_1) if f.endswith('.jpg')]
all_defect_cookies_lvl_2=[f"{dataset_path_defect_2}/{f}" for f in os.listdir(dataset_path_defect_2) if f.endswith('.jpg')]
all_defect_cookies_lvl_3=[f"{dataset_path_defect_3}/{f}" for f in os.listdir(dataset_path_defect_3) if f.endswith('.jpg')]
all_defect_cookies = all_defect_cookies_lvl_1 + all_defect_cookies_lvl_2 + all_defect_cookies_lvl_3  

print("Original is good")
ei_inference(test_good_cookies[0])
print("\nOriginal is anomaly lvl 1")
ei_inference(all_defect_cookies_lvl_1[0])
print("\nOriginal is anomaly lvl 2")
ei_inference(all_defect_cookies_lvl_2[0])
ei_inference( "datasets/cookies_2/anomaly_lvl_2/20240417_142025.jpg")
print("\nOriginal is anomaly lvl 3")
ei_inference(all_defect_cookies_lvl_3[0])

modelfile ../output/fomoad_2.eim
Original is good
{'anomaly': 0.0, 'visual_anomaly_grid': [{'height': 17, 'label': 'anomaly', 'value': 15.509693145751953, 'width': 17, 'x': 0, 'y': 0}, {'height': 17, 'label': 'anomaly', 'value': 10.469318389892578, 'width': 17, 'x': 17, 'y': 0}, {'height': 17, 'label': 'anomaly', 'value': 12.342967987060547, 'width': 17, 'x': 35, 'y': 0}, {'height': 17, 'label': 'anomaly', 'value': 11.255226135253906, 'width': 17, 'x': 53, 'y': 0}, {'height': 17, 'label': 'anomaly', 'value': 11.922293663024902, 'width': 17, 'x': 71, 'y': 0}, {'height': 17, 'label': 'anomaly', 'value': 11.872461318969727, 'width': 17, 'x': 88, 'y': 0}, {'height': 17, 'label': 'anomaly', 'value': 12.051149368286133, 'width': 17, 'x': 106, 'y': 0}, {'height': 17, 'label': 'anomaly', 'value': 11.078412055969238, 'width': 17, 'x': 124, 'y': 0}, {'height': 17, 'label': 'anomaly', 'value': 10.837323188781738, 'width': 17, 'x': 142, 'y': 0}, {'height': 17, 'label': 'anomaly', 'value': 20.29501